In [2]:
categories = ['alt.atheism', 'soc.religion.christian',
              'comp.graphics', 'sci.med']

In [5]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.cross_validation import cross_val_score
from sklearn import tree
from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.externals.six import StringIO
#import matplotlib.pyplot as plt

In [6]:
twenty_train = fetch_20newsgroups(subset='train',
    categories=categories, shuffle=True, random_state=42)

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

In [8]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)
X_train_counts.shape

(2257, 35788)

In [9]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(2257, 35788)

In [10]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train_tfidf, twenty_train.target)

In [11]:
docs_new = ['God is love', 'OpenGL on the GPU is fast']
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

for doc, category in zip(docs_new, predicted):
    print('%r => %s' % (doc, twenty_train.target_names[category]))

'God is love' => soc.religion.christian
'OpenGL on the GPU is fast' => comp.graphics


In [12]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', tree.DecisionTreeClassifier(min_samples_split=5, min_samples_leaf=2)),])

In [13]:
text_clf = text_clf.fit(twenty_train.data, twenty_train.target)

In [14]:
import numpy as np
twenty_test = fetch_20newsgroups(subset='test',
    categories=categories, shuffle=True, random_state=42)
docs_test = twenty_test.data
predicted = text_clf.predict(docs_test)
np.mean(predicted == twenty_test.target)   

0.71438082556591209

In [15]:
from sklearn import metrics
print(metrics.classification_report(twenty_test.target, predicted,
    target_names=twenty_test.target_names))

                        precision    recall  f1-score   support

           alt.atheism       0.64      0.66      0.65       319
         comp.graphics       0.70      0.80      0.75       389
               sci.med       0.70      0.58      0.63       396
soc.religion.christian       0.80      0.81      0.81       398

           avg / total       0.71      0.71      0.71      1502



In [107]:
plt.figure()
plt.clf()

# Plot the training points
plt.scatter(docs_test[:, 0], twenty_train.target[:, 1], c=y, label="data")
#plt.plot(xNew, predict, c="g", label="max_depth=2", linewidth=2)

#plt.xlabel("data")
#plt.ylabel("target")

plt.show()

TypeError: list indices must be integers or slices, not tuple